# Проект по SQL 

## Описание исследования

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

В нашем распоряжении имеются таблицы, содержащие ключевую информацию о книгах, авторах, издательствах, пользовательских оценках и обзорах на книги.

Таблица "books" содержит данные о книгах:

- book_id — идентификатор книги;

- author_id — идентификатор автора;

- title — название книги;

- num_pages — количество страниц;

- publication_date — дата публикации книги;

- publisher_id — идентификатор издателя.

Таблица authors содержит данные об авторах:

- author_id — идентификатор автора;

- author — имя автора.

Таблица publishers содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица ratings содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;

- book_id — идентификатор книги;

- username — имя пользователя, оставившего оценку;

- rating — оценка книги.

Таблица reviews содержит данные о пользовательских обзорах на книги:

- review_id — идентификатор обзора;

- book_id — идентификатор книги;

- username — имя пользователя, написавшего обзор;

- text — текст обзора.

Цель исследования - проведение анализа имеющейся базы данных и формулирование ценностного приложения для нового продукта.

Во время проведения исследования необходимо выполнить следующие задачи:

1) Определить количество книг, вышедших после 01.01.2020 г.;

2) Определить количество обзоров и среднюю оценку для каждой из книг, содержащейся в базе данных;

3) Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, и исключить из анализа брошюры;

4) Определить автора с самой высокой средней оценкой книг — оценка от 50 и более;

5) Определить среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Подключение к базе данных
Для получения доступа к данным необходимо осуществить подключение к базе данных.

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Т.к. работа с базой данных осуществляется не из ее консоли, а через удаленное подключение, необходимо создать переменную engine и сохранить в ней код для предотвращения закрытия удаленного подключения через определенный период бездействия.

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

## Исследование имеющихся в распоряжении таблиц

Далее изучим информацию, содержащуюся в каждой из имеющихся таблиц: 

1) Таблица books: 

С помощью info() посмотрим ключевую информацию о таблице. 

In [4]:
query = '''
            SELECT * 
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [5]:
books.head(5)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таким образом, мы видим, что таблица состоит из 6 столбцов и 1000 строк. 

Далее определим количество уникальных авторов, издателей, минимальное и максимальное количество страниц, самую раннюю и позднюю даты публикации, проверим дубликаты. 

In [6]:
#определим количество уникальных авторов
print('Количество уникальных авторов: ', books['author_id'].nunique())

Количество уникальных авторов:  636


In [7]:
#определим количество уникальных издателей
print('Количество уникальных издателей: ', books['publisher_id'].nunique())

Количество уникальных издателей:  340


In [8]:
#определим количество уникальных книг
print('Количество уникальных книг: ', books['book_id'].nunique())

Количество уникальных книг:  1000


In [9]:
#определим количество уникальных книг
print('Количество уникальных книг: ', books['title'].nunique())

Количество уникальных книг:  999


In [10]:
#определим минимальное и максимальное количество страниц
print('Минимальное количество страниц: ', books['num_pages'].min(), ', максимальное количество страниц: ', books['num_pages'].max())

Минимальное количество страниц:  14 , максимальное количество страниц:  2690


In [11]:
#определим самую раннюю и самую позднюю даты публикации печатного издания
print('Самая ранняя дата публикации: ', books['publication_date'].min(), ', самая поздняя дата публикации: ', books['publication_date'].max())

Самая ранняя дата публикации:  1952-12-01 , самая поздняя дата публикации:  2020-03-31


In [12]:
books.duplicated().sum()

0

Таким образом, мы определили количество уникальных авторов и издателей, максимальное и минимальное количество страниц, самую раннюю и самую позднюю даты публикации, убедились в отсутствии дубликатов в таблице books. 

2) Таблица authors:

In [13]:
query = '''SELECT *
           FROM authors
           '''

authors = pd.io.sql.read_sql(query, con = engine)
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [14]:
authors.head(5)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [15]:
print('Количество уникальных авторов: ', authors['author_id'].nunique())

Количество уникальных авторов:  636


In [16]:
print('Количество уникальных авторов: ', authors['author'].nunique())

Количество уникальных авторов:  636


In [17]:
authors.duplicated().sum()

0

Таблица authors состоит из 2 столбцов и содержит 636 строк. При знакомстве с таблицей books мы также увидели, что количество уникальных авторов в таблицах совпадает, дубликаты в таблице authors отсутствуют.  

3) Таблица publishers: 



In [18]:
query = '''SELECT *
           FROM publishers
           '''

publishers = pd.io.sql.read_sql(query, con = engine)
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [19]:
publishers.head(5)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [20]:
print('Количество уникальных издателей: ', publishers['publisher_id'].nunique())

Количество уникальных издателей:  340


In [21]:
print('Количество уникальных издателей: ', publishers['publisher'].nunique())

Количество уникальных издателей:  340


In [22]:
publishers.duplicated().sum()

0

Таблица authors также как и таблица authors состоит из двух столбцов. Количество уникальных издателей в таблице - 340, как и в таблице books. 

4) Таблица ratings: 

In [23]:
query = '''SELECT *
           FROM ratings
           '''

ratings = pd.io.sql.read_sql(query, con = engine)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [24]:
ratings.head(5)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [25]:
print('Количество оценок: ', ratings['rating_id'].nunique())

Количество оценок:  6456


In [26]:
print('Количество книг: ', ratings['book_id'].nunique())

Количество книг:  1000


In [27]:
print('Количество пользователей, оставивших оценки: ', ratings['username'].nunique())

Количество пользователей, оставивших оценки:  160


In [28]:
print('Минимальный рейтинг: ', ratings['rating'].min(), ', максимальный рейтинг: ', ratings['rating'].max())

Минимальный рейтинг:  1 , максимальный рейтинг:  5


In [29]:
ratings.duplicated().sum()

0

Таблица ratings содержит 4 колонки и 6456 строк. Количество книг такое же, как и в таблице books. В таблице содержится пятибальная шкала оценивания. 

5) Таблица reviews: 



In [30]:
query = '''SELECT *
           FROM reviews
           '''

reviews = pd.io.sql.read_sql(query, con = engine)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [31]:
reviews.head(5)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [32]:
print('Количество обзоров: ', reviews['review_id'].nunique())

Количество обзоров:  2793


In [33]:
print('Количество книг: ', reviews['book_id'].nunique()) 

Количество книг:  994


In [34]:
print('Количество пользователей, написавших обзор: ', reviews['username'].nunique())

Количество пользователей, написавших обзор:  160


In [35]:
print('Количество обзоров: ', reviews['text'].nunique())

Количество обзоров:  2793


In [36]:
reviews.duplicated().sum()

0

Таблица reviews содержит 4 колонки и 2793 строки. Количество пользователей такое же, как и в таблице ratings, однако количество книг меньше - 994.

Таким образом, мы изучили ключевую информацию о таблицах и теперь можем приступить к анализу.

## Анализ базы данных
Далее проведем исследовательский анализ базы данных и найдем ответы на поставленные вопросы.

Сначала определим, какое количество книг вышло после 01.01.2020 г.

In [37]:
query = '''
            SELECT COUNT(book_id) count_of_books
            FROM books 
            WHERE publication_date > '01-01-2000'
        '''
pd.io.sql.read_sql(query, con = engine)

,count_of_books
0,819


Таким образом, после 01.01.2020 г. было выпущено 819 книг.

Далее для каждой книги посчитаем количество обзоров и среднюю оценку.

In [45]:
query = '''
            SELECT b.title AS title, b.book_id AS id, AVG(ratings.rating) AS avg_rating, COUNT(DISTINCT reviews.text) AS reviews_count
            FROM books b
            LEFT JOIN ratings ON b.book_id = ratings.book_id
            LEFT JOIN reviews ON b.book_id = reviews.book_id 
            GROUP BY
                 b.title,
                 b.book_id
            ORDER BY
                 reviews_count DESC
        '''
pd.io.sql.read_sql(query, con = engine)

,title,id,avg_rating,reviews_count
0,Twilight (Twilight #1),948,3.662500,7
1,Water for Elephants,963,3.977273,6
2,The Glass Castle,734,4.206897,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,4.414634,6
4,The Curious Incident of the Dog in the Night-Time,695,4.081081,6
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,3.666667,0
996,The Natural Way to Draw,808,3.000000,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.000000,0
998,Essential Tales and Poems,221,4.000000,0



Далее определим издательство, которое выпустило наибольшее число книг толще 50 страниц.



In [39]:
query = '''
            SELECT b.publisher_id, p.publisher, COUNT(*) count_books
            FROM books b
            LEFT JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY b.publisher_id, p.publisher
            ORDER BY COUNT(*) DESC
            LIMIT 1
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,count_books
0,212,Penguin Books,42


Таким образом, издательством, выпустившим большее количество книг толще 50 страниц, является Penguin Books (42 выпущенные книги).

Далее определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками.

In [40]:
query = '''
            SELECT a.author, AVG(ratings)
            FROM (
            SELECT book_id, AVG(rating) ratings
            FROM ratings
            GROUP BY book_id
            HAVING COUNT(*) > 50
            ) rat
            JOIN (SELECT book_id, author_id FROM books) b ON rat.book_id=b.book_id
            JOIN (SELECT author_id, author FROM authors) a ON b.author_id=a.author_id
            GROUP BY a.author
            ORDER BY avg(ratings) DESC

            
        '''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Таким образом, было выявлено 14 авторов с самой высокой средней оценкой книг:

- авторы со средней оценкой от 4: J.K. Rowling/Mary GrandPré, Markus Zusak/Cao Xuân Việt Khương, J.R.R. Tolkien, Louisa May Alcott, Rick Riordan.

- самая низкая средняя оценка у Stephenie Meyer и John Steinbeck.

Далее посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [41]:
query = '''
            SELECT AVG(count_reviews)
            FROM (
            SELECT username, COUNT(*) count_reviews
            FROM reviews
            WHERE username IN
            (
            SELECT username--, COUNT(*)
            FROM ratings
            GROUP BY username
            HAVING COUNT(*) > 48
            )
            GROUP BY username) rev
        

        '''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.0


Таким образом, среднее количество обзоров от пользователей, которые поставили более 50 оценок - 24.

## Вывод

Таким образом, в ходе анализа было выявлено следующее: 

1) определено, что после 01.01.2020 г. вышло 819 книг;

2) были посчитаны количество обзоров и средняя оценка для каждой из книг. Шкала оценивания книг - от 1 до 5. Наибольшее количество обзоров у книг Twilight, The Road, The Book Thief, The Glass Castle, Water for Elephants, наименьшее количество обзоров - у Naked Empire (Sword of Truth #8), Moo Baa La La La!, Merrick (The Vampire Chronicles #7), Babyville,	Lysistrata;

3) определено, что издательством, выпустившим наибольшее количество книг с 50 и более страницами, является Penguin Books (количество выпущенных книг - 42);

4) определен автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré;

5) определено среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24. 

Таким образом можно порекомендовать владельцам бизнеса следующее: 

1) расширить список книг в приложении от авторов с наивысшим рейтингами;

2) больше сосредоточиться на сотрудничестве с издательством Penguin Books;

3) рассмотреть возможность поощрения пользователей за оставление обзоров на книги (например, предоставление премиум доступа на определенный период или систему скидок на следующие покупки).